<a href="https://colab.research.google.com/github/kangyoon115/blog/blob/gh-pages/%EA%B8%B0%EA%B3%84%ED%95%99%EC%8A%B5_%EB%B6%84%EB%A5%98%EB%AC%B8%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#여행 보험 패키지 상품을 구매할 확률 값을 구하시오
#예측할 값(y): TravelInsurance (여행보험 패지지를 구매 했는지 여부 0:구매안함, 1:구매)
#평가: roc-auc 평가지표

In [ ]:
#예측할 값이 0 또는 1 범주 값으로 구성되어 있기 때문에 '분류문제'로 접근 
#다만, 문제에서 0~ 1사이의 확률 값으로 추출하라 하였기 때문에, 분류모델에서 확률값 그대로 추출 ! (범주확정 X)

In [ ]:
# 1. 데이터 불러오기

import pandas as pd
import numpy as np

dataset = pd.read_csv("여행보험데이터.csv")

In [ ]:
# 2. 데이터 확인 : 변수개수, 행개수, 데이터 구조 등

dataset.head()

,id,Age,Employment Type,GraduateOrNot,AnnualIncome,FamilyMembers,ChronicDiseases,FrequentFlyer,EverTravelledAbroad,TravelInsurance
0,10000,28,Private Sector/Self Employed,Yes,1250000.0,6,1,No,No,0
1,10001,31,Private Sector/Self Employed,Yes,1250000.0,7,1,No,No,0
2,10002,29,Private Sector/Self Employed,Yes,1200000.0,7,0,No,No,1
3,10003,33,Government Sector,Yes,650000.0,6,1,No,No,1
4,10004,28,Private Sector/Self Employed,Yes,800000.0,6,0,No,Yes,1


In [ ]:
dataset.shape #10개의 변수, 총 1,490개 데이터

(1490, 10)

In [ ]:
dataset.info()

# 10개 변수 각각의 이름, null 여부, 데이터타입 확인
# 'AnnualIncome' 변수가 4개의 NaN을 갖고 있는 것을 확인할 수 있음

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1490 entries, 0 to 1489
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   1490 non-null   int64  
 1   Age                  1490 non-null   int64  
 2   Employment Type      1490 non-null   object 
 3   GraduateOrNot        1490 non-null   object 
 4   AnnualIncome         1486 non-null   float64
 5   FamilyMembers        1490 non-null   int64  
 6   ChronicDiseases      1490 non-null   int64  
 7   FrequentFlyer        1490 non-null   object 
 8   EverTravelledAbroad  1490 non-null   object 
 9   TravelInsurance      1490 non-null   int64  
dtypes: float64(1), int64(5), object(4)
memory usage: 116.5+ KB


In [ ]:
# 데이터분석을 할 때 전처리, 모델설계, 학습, 검증도 중요하지만
# 가장 중요한 것은 데이터를 이해하는 것.
# 따라서, 이 데이터가 무슨 데이터인지, 각 컬럼은 무엇을 의미하는지, 변수 간 어떤 연관성이 있을지 생각하는 것이 중요
# 그래야, 수치를 보지 않고도 어느정도 관계를 파악하고 분석을 잘 진행할 수 있음.



# 예시
# 해당 데이터는 '여행 보험 패키지 상품 데이터'임
# 내가 목표로하는 것은 특정 사람이 여행 보험 패키지를 구매할지 안할지 판단하기 위해 '구매할 확률을 계산'하는 것

# -> 이를 통해, 고객정보라는 것을 유추할 수 있고
#    일반적으로 남녀, 나이, 직업, 누구랑 여행가는가 등 기본 고객정보나 여행과 관련된 정보가 있을 수 있을 것이라 생각 가능
#    그리고 어떤 데이터가 구매할 확률과 상관성이 높을지 미리 생각할 수 있음.

# -> 이처럼 도메인을 미리 파악해야, 빈 값을 처리한다든가, 이상치를 처리한다든가, 변수를 통합한다든가 할 때 효과적으로 처리 가능

In [ ]:
# 3. 학습 데이터 카테고리 수 확인
dataset.describe(include="object") #exclude="object" 하면 수치형 변수만 출력

,Employment Type,GraduateOrNot,FrequentFlyer,EverTravelledAbroad
count,1490,1490,1490,1490
unique,2,2,2,2
top,Private Sector/Self Employed,Yes,No,No
freq,1056,1270,1175,1209


In [ ]:
# 4. 결측지 처리하기
dataset.isnull().sum() #결측지 개수 확인

id                     0
Age                    0
Employment Type        0
GraduateOrNot          0
AnnualIncome           4
FamilyMembers          0
ChronicDiseases        0
FrequentFlyer          0
EverTravelledAbroad    0
TravelInsurance        0
dtype: int64

In [ ]:
# 일반적으로 Null 처리는 
# (1) 행삭제(데이터삭제)
# (2) 열삭제(변수삭제)

# (3) 값 대체 -> 평균대체, 중앙값대체, 특정값 대체 등

# 3가지 방법으로 존재하며,
# - Null이 너무 많은 경우엔 열 삭제를,
# - 개수가 별로 없는 경우엔 행 삭제를,
# - 데이터 손실을 막기 위해서는 값 대체를 사용한다.

# 특히, 검증 데이터에 Null이 있으면 값 대체를 사용하여야 한다. 빅분기에서는 왠만하면 값 대체를 사용하는 것이 좋다.


# 추가적으로, 결측치 처리는 데이터셋마다 수많은 방법으로 처리할 수 있는데 가볍게 생각하면 안된다.
# 예를들어, 시계열 데이터의 경우, 시간을 반영하여 대체하는 것이 좋다.(각 요일 평균, 앞의 데이터로 대체, 뒤의 데이터로 대체 등)

In [ ]:
# 결측치 단순 평균 대체 처리
dataset['AnnualIncome'] = dataset['AnnualIncome'].fillna(dataset['AnnualIncome'].mean())

In [ ]:
# 5. 수치 데이터 통계 값 확인
dataset.describe(exclude="object")


# 해당 결과를 보고 상식선에서 가능한 값을 벗어난게 있는지 먼저 확인한다.
# 예를 들어, 나이인데 음수 값이 존재하는지, 100이상의 값이 존재하는지, 소수자리가 존재하는지 등


# (1) 모두 일단, 큰 문제 없어 보임.
# (2) 통계값을 확인해보니 'ChronicDiseases'변수가 0또는 1 이진변수일 수 도 있고, 대부분 0에 해당하는 변수일 것 같아보임
# (3) 'FamilyMembers' 변수는 가족인원수 변수로, 정수일 것으로 보이고, 범주형으로 바꾸기 쉬울 것으로 보임

,id,Age,AnnualIncome,FamilyMembers,ChronicDiseases,TravelInsurance
count,1490.000000,1490.000000,1.490000e+03,1490.000000,1490.000000,1490.000000
mean,10744.500000,29.600000,9.311238e+05,4.755705,0.280537,0.352349
std,430.270264,2.887829,3.759814e+05,1.603613,0.449412,0.477862
min,10000.000000,25.000000,3.000000e+05,2.000000,0.000000,0.000000
25%,10372.250000,28.000000,6.000000e+05,4.000000,0.000000,0.000000
50%,10744.500000,29.000000,9.000000e+05,5.000000,0.000000,0.000000
75%,11116.750000,32.000000,1.250000e+06,6.000000,1.000000,1.000000
max,11489.000000,35.000000,1.800000e+06,9.000000,1.000000,1.000000


In [ ]:
print(dataset["ChronicDiseases"].value_counts()) #실제로 이진변수인 것 확인 

0    1072
1     418
Name: ChronicDiseases, dtype: int64


In [ ]:
print(dataset["FamilyMembers"].value_counts())

4    385
5    311
3    281
6    225
7    139
2     68
9     41
8     40
Name: FamilyMembers, dtype: int64


In [ ]:
#6. 종속변수 분포 확인 (범주형 종속변수라면 불균형이 있는지, 수치형 종속변수라면 쏠림현상 있는지 등)
dataset["TravelInsurance"].value_counts()

0    965
1    525
Name: TravelInsurance, dtype: int64

In [ ]:
# 7. 학습 데이터만 가지고 종속변수와 독립변수 간 상관관계 확인 -> 중요변수 확인 #다만, 종속변수가 연속형 변수여야 한다.


numeric_data = dataset.select_dtypes(exclude="object")
numeric_data.loc[:, numeric_data.columns != 'id'].corr()

# 'AnnualIncome'변수의 상관성이 높은 것 확인
# 독립변수간 상관관계 없는 것으로 보임 (다중공선성 x)
# 사실 범주형 종속변수라 이렇게 보는 것 한계 O

,Age,AnnualIncome,FamilyMembers,ChronicDiseases,TravelInsurance
Age,1.000000,-0.005363,0.024422,0.000103,0.073487
AnnualIncome,-0.005363,1.000000,-0.023881,-0.009454,0.382361
FamilyMembers,0.024422,-0.023881,1.000000,0.023405,0.095751
ChronicDiseases,0.000103,-0.009454,0.023405,1.000000,0.027263
TravelInsurance,0.073487,0.382361,0.095751,0.027263,1.000000


In [ ]:
# 8. 수치 데이터 이상치 확인 및 대체


# 이상치 탐색은 
# (1) Q1 - (IQR * 1.5)
#     Q3 + (IQR * 1.5)
    

# (2) 평균 +- 표준편차 * 3

# 방식으로 주로 해결함

In [ ]:
numeric = ["Age", "AnnualIncome", "FamilyMembers", "ChronicDiseases"]

for i in numeric:
    # Q1 = dataset[i].quantile(0.25)
    # Q3 = dataset[i].quantile(0.75)
    # iqr = (Q3-Q1) * 1.5
    # min_value = Q1-iqr
    # max_value = Q3+iqr

    means = dataset[i].mean()
    min_value = means-dataset[i].std()*3
    max_value = means+dataset[i].std()*3

    for j in range(len(dataset)):
        if dataset[i][j] < min_value:
            dataset[i][j] = min_value
            print("Ok")
        
        elif dataset[i][j] > max_value:
            dataset[i][j] = max_value
            print("Ok")


In [ ]:
# 9. 데이터 변환(범주통합 등)을 해야하는 변수들이 있는지 확인 (종속변수 예측/분류에 의미가 있어야 함)

dataset["Age_c"] = pd.qcut(dataset["Age"], q=5)
print(dataset["Age_c"].value_counts())

(27.0, 28.0]      370
(30.0, 33.0]      361
(24.999, 27.0]    329
(33.0, 35.0]      231
(28.0, 30.0]      199
Name: Age_c, dtype: int64


In [ ]:
#10. 범주형 변수 데이터 레이블링 진행 (모델에 input 시키기 위함)

from sklearn.preprocessing import LabelEncoder

cols = ["Employment Type", "GraduateOrNot", "FrequentFlyer", "EverTravelledAbroad", "Age_c"]
le = LabelEncoder()

for col in cols:
    dataset[col] = le.fit_transform(dataset[col])

In [ ]:
dataset.head(3)

#del dataset["id"]
#del dataset["Age"]

,id,Age,Employment Type,GraduateOrNot,AnnualIncome,FamilyMembers,ChronicDiseases,FrequentFlyer,EverTravelledAbroad,TravelInsurance,Age_c
0,10000,28,1,1,1250000.0,6,1,0,0,0,1
1,10001,31,1,1,1250000.0,7,1,0,0,0,3
2,10002,29,1,1,1200000.0,7,0,0,0,1,2


In [ ]:
# 11. 독릭변수와 종속변수 분리
# v_name = dataset.columns
v_name = ['Employment Type', 'GraduateOrNot', 'AnnualIncome',
       'FamilyMembers', 'ChronicDiseases', 'FrequentFlyer',
       'EverTravelledAbroad', 'Age_c']

target_name = 'TravelInsurance'

target = dataset[target_name]
dataset = dataset[v_name]

In [ ]:
#12. train-validation 분리

from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(dataset, target, test_size=0.2, random_state=0)

In [ ]:
# 모델 선택 및 학습
from sklearn.metrics import accuracy_score

from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier

model = LogisticRegression()
# model = SVC(kernel="rbf", C=10)
# model = RandomForestClassifier(max_depth=2)
# model= XGBClassifier(max_depth=2, random_state=1)


model.fit(train_x, train_y)
pred = model.predict(test_x)
result = accuracy_score(test_y, pred)

print(result)

0.6375838926174496
